**Table de contenido**

- [Introducción](#Introduccion)
- [Requisitos](#Requisitos)
- [Lectura de datos](#Lectura-de-datos)
- [Preprocesamiento](#Preprocesamiento)
    - [Limpiar texto](#Limpiar-texto)

# Introduccion

Este cuaderno, lo usaremos para agregar las coordenadas a las ciudades del archivo Dataset_Entregas. La idea es poder calcular la distancia que hay entre la ciudad de origen y la ciudad de destino, esto con el propósito de tener más variables predictoras para el modelo que se encarga de predecir si una entrega sera puntual.

# Requisitos

El siguiente cuaderno fue desarrollado en un entorno de anaconda con las siguientes características:

- Python 3.12.11
- Pandas version: 2.3.1
- NumPy version: 2.3.1
- Haversine version: 2.9.0

Descargar el archivo CO, del siguiente link:

- https://download.geonames.org/export/zip/CO.zip

# Lectura de datos

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

project_root = next(p for p in Path.cwd().parents if (p / 'data').exists()) 
file_path = lambda file : os.path.join(project_root,'data',file)
data_entregas = pd.read_csv(file_path('DataSet_Entregas.csv'))
data_entregas.head()

,ID,Ciudad_Origen,Ciudad_Destino,Terminal_Origen,Terminal_Destino,Producto,Cliente,Fecha_Recogida,Fecha_Entrega,Dias_Ofrecidos,Dias_Transcurridos,Peso,Peso_Volumen,Unidades
0,11,Bucaramanga (Stder),San Juan Del Cesar (Guaj),6. Bucaramanga,20. Valledupar,Paquetería,AB-JABEEHEBI,2023-12-26,2024-01-01,3,4,1.0,1.00,1
1,36,Cartagena (Bol),Santa Marta (Mg/Lena),9. Turbaco,22. Santa Marta,Paquetería,AB-IAFAADGCG,2023-12-29,2024-01-02,1,2,1.0,1.00,1
2,43,Cerrito (Stder),Bucaramanga (Stder),6. Bucaramanga,6. Bucaramanga,Paquetería,AB-DFCHBFHHB,2023-12-30,2024-01-02,1,1,2.0,0.01,1
3,48,Bucaramanga (Stder),Monteria (Cord),6. Bucaramanga,14. Monteria,Paquetería,AH-JABDDDAAH,2023-12-26,2024-01-02,2,5,1.0,0.40,1
4,49,Pereira (Rs),La Hormiga - Valle Guamuez (P/Mayo),4. Pereira,23. Neiva,Paquetería,AB-DFCHBFHHB,2023-12-26,2024-01-02,5,5,2.0,0.01,1


In [2]:
file_path = lambda file : os.path.join(project_root,'data',file)

columnas = [
    "country_code", 
    "postal_code", 
    "place_name", 
    "admin_name1", 
    "admin_code1",
    "admin_name2",
    "admin_code2",
    "admin_name3",
    "admin_code3",
    "latitude",
    "longitude",
    "accuracy"
]
colombia = pd.read_csv(
    file_path("CO.txt"),
    sep="\t",
    header=None,
    names=columnas,
    dtype=str,
    encoding="utf-8"
)
colombia.head()

,country_code,postal_code,place_name,admin_name1,admin_code1,admin_name2,admin_code2,admin_name3,admin_code3,latitude,longitude,accuracy
0,CO,910001,Leticia,Amazonas,01,Leticia,91001,NaN,NaN,-4.2153,-69.9406,4
1,CO,910007,Leticia,Amazonas,01,Leticia,91001,NaN,NaN,-4.2153,-69.9406,4
2,CO,910008,Leticia,Amazonas,01,Leticia,91001,NaN,NaN,-4.2153,-69.9406,4
3,CO,913010,El Encanto,Amazonas,01,El Encanto,91263,NaN,NaN,-1.7477,-73.2083,4
4,CO,913017,El Encanto,Amazonas,01,El Encanto,91263,NaN,NaN,-1.7477,-73.2083,4


In [3]:
# Convertir lat y lon a numérico
colombia["latitude"] = pd.to_numeric(colombia["latitude"], errors="coerce")
colombia["longitude"] = pd.to_numeric(colombia["longitude"], errors="coerce")

# Preprocesamiento

Vamos a consultar los estadísticos básicos de dataframe de entregas, la cantidad de resitros, verificar si hay valores faltantes y duplicados,  

In [4]:
print(f"la cantidad de resgistros que tiene el dataframe son: {data_entregas.shape[0]}")
print(f"La cantidad de características son: {data_entregas.shape[1]}")

la cantidad de resgistros que tiene el dataframe son: 5158113
La cantidad de características son: 14


In [5]:
pd.set_option('display.float_format', '{:.2f}'.format)
data_entregas.describe()

,ID,Dias_Ofrecidos,Dias_Transcurridos,Peso,Peso_Volumen,Unidades
count,5158113.00,5158113.00,5158113.00,5158113.00,5158113.00,5158113.00
mean,2579057.00,2.14,2.93,8.84,14.64,1.21
std,1489019.11,1.33,4.85,142.38,88.67,5.77
min,1.00,1.00,-1.00,0.01,0.00,1.00
25%,1289529.00,1.00,1.00,1.00,0.57,1.00
50%,2579057.00,2.00,2.00,1.00,1.00,1.00
75%,3868585.00,3.00,3.00,7.00,12.00,1.00
max,5158113.00,17.00,724.00,249615.00,32986.80,1817.00


Ok. Puedo observar que en la variable **Dias_Transcurridos** el valor mínimo es un valor negativo (-1.00) y el valor máximo de la variable `Dias_transcurridos` es de 724.00, indicando muy posiblemente la presencia de valores atípicos. En el caso de la variable `Peso_Volumen`, esta presenta una desviación estandar de 88.67, indicando que los productos varian mucho de Peso_volumen.

Veamos si este dataset tiene datos faltantes.

In [6]:
data_entregas.isnull().sum()

ID                    0
Ciudad_Origen         0
Ciudad_Destino        0
Terminal_Origen       0
Terminal_Destino      0
Producto              0
Cliente               0
Fecha_Recogida        0
Fecha_Entrega         0
Dias_Ofrecidos        0
Dias_Transcurridos    0
Peso                  0
Peso_Volumen          0
Unidades              0
dtype: int64

Ok, hay valores faltantes, pero en el cuaderno de modelado, aplicaremos técnicas para poder tratar con estos datos.

In [7]:
print(f"Duplicados (sin contar la primera aparición): {data_entregas.duplicated().sum()}")

Duplicados (sin contar la primera aparición): 0


Esto es positivo; no hay registros duplicados. Ahora, examinemos qué tipos de datos contiene el dataframe.

In [8]:
data_entregas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5158113 entries, 0 to 5158112
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ID                  int64  
 1   Ciudad_Origen       object 
 2   Ciudad_Destino      object 
 3   Terminal_Origen     object 
 4   Terminal_Destino    object 
 5   Producto            object 
 6   Cliente             object 
 7   Fecha_Recogida      object 
 8   Fecha_Entrega       object 
 9   Dias_Ofrecidos      int64  
 10  Dias_Transcurridos  int64  
 11  Peso                float64
 12  Peso_Volumen        float64
 13  Unidades            int64  
dtypes: float64(2), int64(4), object(8)
memory usage: 550.9+ MB


Perfecto, lo que aremos ahora será eliminar los registros negativos de la columnas "Dias_Transcurridos" (días hábiles entre la fecha de recogida y la fecha de entrega) eliminaremos los registros negativos, pues un valor negativo implicaría que la fecha de entrega es anterior a la fecha de recogida, lo cual no es posible en el proceso real.

In [9]:
data_entregas = data_entregas[
    (data_entregas['Dias_Transcurridos'] >= 0) 
]

## Limpiar texto

Aqui lo aremos será obtener los nombres de las ciudades y departamentos

In [10]:
def extraer_ciudad(terminal):
    """
    Función para extraer el nombre de la ciudad de un terminal.
    
    Parámetros:
    terminal (str): Terminal en formato de cadena.
    
    Retorna:
    str: Nombre de la ciudad o None si no es válido.
    """
    if isinstance(terminal, str) and '.' in terminal:
        return str(terminal.split('.')[1]).strip()
    return None 
data_entregas['Ciudad_O'] = data_entregas['Terminal_Origen'].apply(extraer_ciudad)
data_entregas['Ciudad_D'] = data_entregas['Terminal_Destino'].apply(extraer_ciudad)

Obtengamos los departamento

In [11]:
import re
def extraer_departamento(ciudad):
    """
    Extrae el texto dentro de paréntesis (el departamento).
    """
    m = re.search(r"\((.*?)\)", str(ciudad))
    if m:
        return m.group(1).strip()
    return None

data_entregas['Dept_O'] = data_entregas['Ciudad_Origen'].apply(extraer_departamento)
data_entregas['Dept_D'] = data_entregas['Ciudad_Destino'].apply(extraer_departamento)

Perfecto, consultemos cuantos departamentos tenemos en nuesto sed de datos.

In [12]:
data_entregas['Dept_O'].unique()

array(['Stder', 'Bol', 'Rs', 'N/Stder', 'Boy', 'Cdas', 'Hla', 'Meta',
       'Tol', 'Ant', 'Sucre', 'Mg/Lena', 'Qdio', 'Nar', 'Caq', 'Guaj',
       'Valle', 'Cord', 'Ces', 'G/Viare', 'C/Marca', 'C/Nare', 'V/Pes',
       'P/Mayo', 'Choco', None, 'Guai', 'Vich', 'Arau', 'Cau'],
      dtype=object)

Ok, esto indica que muy posiblemente estas son nuestras abreviaciones.

In [13]:
map_departamentos = {
    'Stder': 'Santander',
    'Bol': 'Bolivar',
    'Rs': 'Risaralda',
    'N/Stder': 'Norte de Santander',
    'Boy': 'Boyaca',
    'Cdas': 'Caldas',
    'Hla': 'Huila',
    'Meta': 'Meta',
    'Tol': 'Tolima',
    'Ant': 'Antioquia',
    'Sucre': 'Sucre',
    'Mg/Lena': 'Magdalena',
    'Qdio': 'Quindio',
    'Nar': 'Nariño',
    'Caq': 'Caqueta',
    'Guaj': 'La Guajira',
    'Valle': 'Valle del Cauca',
    'Cord': 'Cordoba',
    'Ces': 'Cesar',
    'G/Viare': 'Guaviare',
    'C/Marca': 'Cundinamarca',
    'C/Nare': 'Casanare',
    'V/Pes': 'Vaupes',
    'P/Mayo': 'Putumayo',
    'Choco': 'Choco',
    None: None,
    'Guai': 'Guainia',
    'Vich': 'Vichada',
    'Arau': 'Arauca',
    'Cau': 'Cauca'
}

data_entregas['Dept_O'] = data_entregas['Dept_O'].map(map_departamentos)
data_entregas['Dept_D'] = data_entregas['Dept_D'].map(map_departamentos)
data_entregas = data_entregas.drop(columns=['Ciudad_Origen','Ciudad_Destino'])


In [14]:
data_entregas.head()

,ID,Terminal_Origen,Terminal_Destino,Producto,Cliente,Fecha_Recogida,Fecha_Entrega,Dias_Ofrecidos,Dias_Transcurridos,Peso,Peso_Volumen,Unidades,Ciudad_O,Ciudad_D,Dept_O,Dept_D
0,11,6. Bucaramanga,20. Valledupar,Paquetería,AB-JABEEHEBI,2023-12-26,2024-01-01,3,4,1.00,1.00,1,Bucaramanga,Valledupar,Santander,La Guajira
1,36,9. Turbaco,22. Santa Marta,Paquetería,AB-IAFAADGCG,2023-12-29,2024-01-02,1,2,1.00,1.00,1,Turbaco,Santa Marta,Bolivar,Magdalena
2,43,6. Bucaramanga,6. Bucaramanga,Paquetería,AB-DFCHBFHHB,2023-12-30,2024-01-02,1,1,2.00,0.01,1,Bucaramanga,Bucaramanga,Santander,Santander
3,48,6. Bucaramanga,14. Monteria,Paquetería,AH-JABDDDAAH,2023-12-26,2024-01-02,2,5,1.00,0.40,1,Bucaramanga,Monteria,Santander,Cordoba
4,49,4. Pereira,23. Neiva,Paquetería,AB-DFCHBFHHB,2023-12-26,2024-01-02,5,5,2.00,0.01,1,Pereira,Neiva,Risaralda,Putumayo


Excelente, ahora necesitamos extraer las ciudades únicas junto con su respectivo departamento. Esto nos permitirá obtener un archivo más compacto y evitar posibles cuellos de botella.

In [15]:
ciudades_unicas = data_entregas[['Ciudad_O', 'Dept_O']].drop_duplicates()
ciudades_unicas = ciudades_unicas[~ciudades_unicas['Dept_O'].isna()]
ciudades_unicas.head()

,Ciudad_O,Dept_O
0,Bucaramanga,Santander
1,Turbaco,Bolivar
4,Pereira,Risaralda
6,Cucuta,Norte de Santander
9,Tunja,Boyaca


Perfecto, ahora si obtengamos las coordenadas de cada ciudad.

In [16]:
latitudes = {ciudad: colombia.loc[colombia['place_name'] == ciudad, 'latitude'].values[0] 
             for ciudad in ciudades_unicas['Ciudad_O']}
longitudes = {ciudad: colombia.loc[colombia['place_name'] == ciudad, 'longitude'].values[0] 
             for ciudad in ciudades_unicas['Ciudad_O']}

In [17]:
ciudades_unicas['Latitud'] = ciudades_unicas['Ciudad_O'].map(latitudes)
ciudades_unicas['Longitud'] = ciudades_unicas['Ciudad_O'].map(longitudes)
ciudades_unicas.head()

,Ciudad_O,Dept_O,Latitud,Longitud
0,Bucaramanga,Santander,7.13,-73.12
1,Turbaco,Bolivar,10.33,-75.41
4,Pereira,Risaralda,4.81,-75.70
6,Cucuta,Norte de Santander,7.89,-72.51
9,Tunja,Boyaca,5.54,-73.37


In [18]:
ciudades_unicas.isnull().sum()

Ciudad_O    0
Dept_O      0
Latitud     0
Longitud    0
dtype: int64

perfecto, ahora si agremos esas coordenadas al dataframe general.

In [19]:
data_entregas = data_entregas[~data_entregas['Dept_O'].isna()]
data_entregas['Latitud_O'] = data_entregas['Ciudad_O'].map(latitudes)
data_entregas['Longitud_O'] = data_entregas['Ciudad_O'].map(longitudes)
data_entregas['Latitud_D'] = data_entregas['Ciudad_D'].map(latitudes)
data_entregas['Longitud_D'] = data_entregas['Ciudad_D'].map(longitudes)
data_entregas = data_entregas.dropna()

In [20]:
data_entregas.isnull().sum()

ID                    0
Terminal_Origen       0
Terminal_Destino      0
Producto              0
Cliente               0
Fecha_Recogida        0
Fecha_Entrega         0
Dias_Ofrecidos        0
Dias_Transcurridos    0
Peso                  0
Peso_Volumen          0
Unidades              0
Ciudad_O              0
Ciudad_D              0
Dept_O                0
Dept_D                0
Latitud_O             0
Longitud_O            0
Latitud_D             0
Longitud_D            0
dtype: int64

Perfecto, ahora calculemos la distancia que hay entre origen y destino.

In [31]:
from haversine import haversine

#coord_origen = (data_entregas['Latitud_O'].values, data_entregas['Longitud_O'].values)
#coord_destino = (data_entregas['Latitud_D'].values, data_entregas['Longitud_D'].values)
distancias = []
for index, row in data_entregas.iterrows():
    coord_origen = (row['Latitud_O'], row['Longitud_O'])
    coord_destino = (row['Latitud_D'], row['Longitud_D'])
    distancia = haversine(coord_origen, coord_destino)
    distancias.append(distancia)

data_entregas['distancia_km'] = distancias

Ahora, eliminemos las coordenadas, pues ya no las necesitamos

In [33]:
data_entregas = data_entregas.drop(columns=['Ciudad_O','Ciudad_D','Latitud_O','Longitud_O','Latitud_D','Longitud_D'],axis=1)

In [34]:
data_entregas.head()

,ID,Terminal_Origen,Terminal_Destino,Producto,Cliente,Fecha_Recogida,Fecha_Entrega,Dias_Ofrecidos,Dias_Transcurridos,Peso,Peso_Volumen,Unidades,Dept_O,Dept_D,distancia_km
0,11,6. Bucaramanga,20. Valledupar,Paquetería,AB-JABEEHEBI,2023-12-26,2024-01-01,3,4,1.00,1.00,1,Santander,La Guajira,371.43
1,36,9. Turbaco,22. Santa Marta,Paquetería,AB-IAFAADGCG,2023-12-29,2024-01-02,1,2,1.00,1.00,1,Bolivar,Magdalena,166.76
2,43,6. Bucaramanga,6. Bucaramanga,Paquetería,AB-DFCHBFHHB,2023-12-30,2024-01-02,1,1,2.00,0.01,1,Santander,Santander,0.00
3,48,6. Bucaramanga,14. Monteria,Paquetería,AH-JABDDDAAH,2023-12-26,2024-01-02,2,5,1.00,0.40,1,Santander,Cordoba,353.62
4,49,4. Pereira,23. Neiva,Paquetería,AB-DFCHBFHHB,2023-12-26,2024-01-02,5,5,2.00,0.01,1,Risaralda,Putumayo,214.69


In [36]:
dir_save_data = project_root/'data/processed'
data_entregas.to_parquet(dir_save_data/ 'Dataset_cumplimiento de entregas_dis.parquet', index=False)